# 本周大綱
+ selenium 爬蟲
+ 等待連線
+ 定期執行
+ 驗證碼
+ 爬蟲的智慧財產權問題
+ 資訊安全
+ 好用模組推薦
+ 其他學習資源

不用抄，下課之前會給這份檔案

## 課後練習
抓取台灣證券交易所最新訊息中，最近一個月的配息資訊

https://www.twse.com.tw/zh/products/securities/etf/news.html

1. 公告標題中包含「分配收益資訊」
2. 挑選今天往前30天的資料（年份是民國要怎麼改成西元？如果今天是2025年4月21日，怎麼往前30天？）
3. 點進網址之後抓取配息金額（配息金額在不同的標的是否會出現在不同位置？）
4. 取得除息交易日並計算出最後買進日

In [14]:
# 匯入模組

from os import listdir
from time import sleep
from datetime import datetime, timedelta
from random import random
import json
import requests
from bs4 import BeautifulSoup

chrome_version = listdir(r'C:\Program Files (x86)\Google\Chrome\Application')[0].split('.')[0]
header = {'User-Agent': f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version}.0.0.0 Safari/537.36'}


In [15]:
url = 'https://www.twse.com.tw/rwd/zh/ETFNews/news?response=json' # 刪掉流水號也可以
r = requests.get(url, headers=header)
data = json.loads(r.text)
data

{'status': 'ok',
 'data': [['2025-06-05T09:09:37Z',
   '期元大S&P原油反1(00673R) 召集受益人大會資訊-公告日期 114年06月05日',
   'https://mopsov.twse.com.tw/mops/web/ajax_t108sb16?step=2&firstin=1&TYPEK=all&SKIND=E&SEQ_NO=1&DATE1=20250605&COMP=A00005',
   None],
  ['2025-06-05T07:51:23Z',
   '大華優利高填息30(00918) 分配收益資訊-公告日期 114年06月05日',
   'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250605&COMP=A00049',
   None],
  ['2025-06-03T05:34:11Z',
   '兆豐龍頭等權重(00921) 分配收益資訊-公告日期 114年06月03日',
   'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250603&COMP=A00001',
   None],
  ['2025-06-03T01:18:55Z',
   '復華台灣科技優息(00929) 分配收益資訊-公告日期 114年06月03日',
   'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250603&COMP=A00022',
   None],
  ['2025-06-02T06:46:59Z',
   '凱基台灣AI50(00952) 分配收益資訊-公告日期 114年06月02日',
   'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&

In [25]:
limit_date = datetime.now() - timedelta(days=30)

dividend_data = {}
for i in data['data']:
    # print(i)
    if '分配收益資訊' in i[1]:
        announcement_time = datetime.strptime(i[0], '%Y-%m-%dT%H:%M:%SZ')
        if announcement_time > limit_date:
            target_name = i[1].split()[0]
            # 只取還沒有資料的
            if dividend_data.get(target_name):
                continue
            else:
                dividend_data[target_name] = i[2]
print(dividend_data)

{'大華優利高填息30(00918)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250605&COMP=A00049', '兆豐龍頭等權重(00921)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250603&COMP=A00001', '復華台灣科技優息(00929)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250603&COMP=A00022', '凱基台灣AI50(00952)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=8&DATE1=20250602&COMP=A00041', '凱基美國非投等債(00945B)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=2&DATE1=20250602&COMP=A00041', '中信亞太高股息(00964)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=10&DATE1=20250602&COMP=A00026', '凱基優選高股息30(00915)': 'https://mopsov.twse.com.tw/mops/web/ajax_t108sb19?step=2&firstin=1&TYPEK=all&SKIND=F&SEQ_NO=1&DATE1=20250602&COMP=A0004

In [27]:
link = list(dividend_data.values())[0]
r = requests.get(link, headers=header)
soup = BeautifulSoup(r.text, 'html.parser')
soup


<html>
<head>
<title>公開資訊觀測站</title>
<link href="css/css2.css" media="Screen" rel="stylesheet" type="text/css"/>
<script src="js/mops2.js" type="text/javascript"></script>
</head>
<body>
<div id="div01">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<center>
<table align="center" class="noBorder"><tr><td class="reportName">
分配收益公告
</td></tr></table>
<!--CDATE ==== 20250605-->
<!--if20230215_1031 ==== false-->
<!--KIND ==== 2-->
<!--FLAG ==== 1-->
<table class="noBorder"><tr><td class="compName">公司代號：A00049　公司名稱：大華銀投信
</td></tr><tr><td class="reportCont">交易證券代號：00918　交易證券名稱：大華優利高填息30
</td></tr><tr><td class="reportCont">類型：指數股票型 為 ●上市 ○上櫃 買賣</td></tr></table>
<table class="hasBorder" width="90%">
<tr><th class="tblHead" colspan="3" style="text-align:left !important;" width="40%">一、公告序號</th><td align="left" class="odd">1</td></tr>
<tr><th class="tblHead" colspan="3" style="text-align:left !important;">二、主旨</th>
<td align="left" class="even">公告本公司經理之「大華銀台灣優選股利高填息 3

In [31]:
from re import search

last_buying_day, amount = '', ''

for tr in soup.select('table tr'):
    # 最後買進日
    if not last_buying_day:
        if '除息交易日' in tr.text:
            exdividend_trading_day = tr.select_one('td').text
            exdividend_trading_day = str(int(exdividend_trading_day.split('年')[0]) + 1911) + '年' + exdividend_trading_day.split('年')[1]
            last_buying_day = datetime.strptime(exdividend_trading_day, '%Y年%m月%d日') - timedelta(days=1)

    # 配息金額
    if not amount:
        for td in tr.select('td'):
            if search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text):
                amount = search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text).group().strip()

print(last_buying_day.date())
print(amount)

2025-06-18
0.70


In [ ]:
# 整理上面的程式碼

from os import listdir
from time import sleep
from datetime import datetime, timedelta
from random import random
import json
from re import search
import requests
from bs4 import BeautifulSoup

chrome_version = listdir(r'C:\Program Files (x86)\Google\Chrome\Application')[0].split('.')[0]
header = {'User-Agent': f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version}.0.0.0 Safari/537.36'}

url = 'https://www.twse.com.tw/rwd/zh/ETFNews/news?response=json' # 刪掉流水號也可以
r = requests.get(url, headers=header)
data = json.loads(r.text)

limit_date = datetime.now() - timedelta(days=30)

dividend_data = {}
for i in data['data']:
    if '分配收益資訊' in i[1]:
        announcement_time = datetime.strptime(i[0], '%Y-%m-%dT%H:%M:%SZ')
        if announcement_time > limit_date:
            target_name = i[1].split()[0]

            # 只取還沒資料的
            if dividend_data.get(target_name):
                continue
            else:
                r = requests.get(i[2], headers=header)

                soup = BeautifulSoup(r.text, 'html.parser')

                # 抓取除息資料
                last_buying_day, amount = '', ''
                for tr in soup.select('table tr'):
                    # 最後買進入
                    if '（五）除息交易日' in tr.text:
                        exdividend_trading_day = tr.select_one('td').text
                        exdividend_trading_day = str(int(exdividend_trading_day.split('年')[0]) + 1911) + '年' + exdividend_trading_day.split('年')[1]
                        last_buying_day = datetime.strptime(exdividend_trading_day, '%Y年%m月%d日') - timedelta(days=1)

                    # 配息金額
                    if not amount:
                        for td in tr.select('td'):
                            if search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text):
                                amount = search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text).group().strip()

                dividend_data[target_name] = [float(amount), last_buying_day.date()]
            sleep(1+random()*2)

        else:
            break

print(dividend_data)

{'大華優利高填息30(00918)': [0.7, datetime.date(2025, 6, 18)], '兆豐龍頭等權重(00921)': [0.12, datetime.date(2025, 6, 16)], '復華台灣科技優息(00929)': [0.07, datetime.date(2025, 6, 17)], '凱基台灣AI50(00952)': [0.02, datetime.date(2025, 6, 16)], '凱基美國非投等債(00945B)': [0.1, datetime.date(2025, 6, 16)], '中信亞太高股息(00964)': [0.043, datetime.date(2025, 6, 16)], '凱基優選高股息30(00915)': [0.21, datetime.date(2025, 6, 16)], '永豐優息存股(00907)': [0.011, datetime.date(2025, 6, 23)], '群益優選非投等債(00953B)': [70.0, datetime.date(2025, 6, 3)], '中信全球高股息(00963)': [0.055, datetime.date(2025, 6, 16)], '中信成長高股息(00934)': [0.028, datetime.date(2025, 6, 16)], '群益科技高息成長(00946)': [0.025, datetime.date(2025, 6, 3)], '中信綠能及電動車(00896)': [0.4, datetime.date(2025, 6, 16)], '台新永續高息中小(00936)': [0.03, datetime.date(2025, 6, 15)], '富邦特選高股息30(00900)': [0.1, datetime.date(2025, 6, 16)], '第一金工業30(00728)': [0.771, datetime.date(2025, 6, 18)], '富邦臺灣優質高息(00730)': [0.11, datetime.date(2025, 6, 16)], '國泰全球品牌50(00916)': [0.25, datetime.date(2025, 6, 16)], '群益台灣精選高息(0

In [32]:
for i in dividend_data:
    print(i ,dividend_data[i])

大華優利高填息30(00918) [0.7, datetime.date(2025, 6, 18)]
兆豐龍頭等權重(00921) [0.12, datetime.date(2025, 6, 16)]
復華台灣科技優息(00929) [0.07, datetime.date(2025, 6, 17)]
凱基台灣AI50(00952) [0.02, datetime.date(2025, 6, 16)]
凱基美國非投等債(00945B) [0.1, datetime.date(2025, 6, 16)]
中信亞太高股息(00964) [0.043, datetime.date(2025, 6, 16)]
凱基優選高股息30(00915) [0.21, datetime.date(2025, 6, 16)]
永豐優息存股(00907) [0.011, datetime.date(2025, 6, 23)]
群益優選非投等債(00953B) [70.0, datetime.date(2025, 6, 3)]
中信全球高股息(00963) [0.055, datetime.date(2025, 6, 16)]
中信成長高股息(00934) [0.028, datetime.date(2025, 6, 16)]
群益科技高息成長(00946) [0.025, datetime.date(2025, 6, 3)]
中信綠能及電動車(00896) [0.4, datetime.date(2025, 6, 16)]
台新永續高息中小(00936) [0.03, datetime.date(2025, 6, 15)]
富邦特選高股息30(00900) [0.1, datetime.date(2025, 6, 16)]
第一金工業30(00728) [0.771, datetime.date(2025, 6, 18)]
富邦臺灣優質高息(00730) [0.11, datetime.date(2025, 6, 16)]
國泰全球品牌50(00916) [0.25, datetime.date(2025, 6, 16)]
群益台灣精選高息(00919) [0.72, datetime.date(2025, 6, 16)]
野村日本動能高息(00972) [0.05, datetime.d

## 進度條
有時候爬蟲要爬很多網頁，會執行很久，如果沒有在某些地方寫個 print 的話，會不知道爬蟲到底有沒有在正常運作，除了 print 之外，可以用進度條模組 tqdm 讓你可以看出來爬蟲執行的概況。

In [60]:
# 由於 tqdm 不是內建的，需要安裝
! pip install tqdm

In [34]:
# 匯入模組
from tqdm import tqdm

進度條有兩種用法，第一種是包在 for loop 外面，例如
```python
for i in tqdm(range(12)):
    執行一些東西
```

第二種是包在 while loop 外面，這種寫法就比較複雜一點：
```python
pbar = tqdm(total=12)
i = 0
while i < 12:
    執行一些東西
    i += 1
    pbar.update(1) # 分子+1

pbar.close() # 關閉進度條
```
在執行的時候，會顯示一個進度條，包含目前進度、已執行時間、預計剩餘時間、每個項目幾秒或每秒幾個項目

In [35]:
# 隨意舉例
for i in tqdm(range(12)):
    sleep(0.1)

# [已執行時間<預計總時長, 浮點數 項目/每秒]
# 比較慢的時候會顯示 s/it，代表每個項目幾秒

100%|██████████| 12/12 [00:01<00:00,  9.51it/s]


In [36]:
# 再跑一次

from os import listdir
from time import sleep
from datetime import datetime, timedelta
from random import random
import json
from re import search
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

chrome_version = listdir(r'C:\Program Files (x86)\Google\Chrome\Application')[0].split('.')[0]
header = {'User-Agent': f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version}.0.0.0 Safari/537.36'}

url = 'https://www.twse.com.tw/rwd/zh/ETFNews/news?response=json' # 刪掉流水號也可以
r = requests.get(url, headers=header)
data = json.loads(r.text)

limit_date = datetime.now() - timedelta(days=30)

dividend_data = {}
for i in tqdm(data['data']):
    if '分配收益資訊' in i[1]:
        announcement_time = datetime.strptime(i[0], '%Y-%m-%dT%H:%M:%SZ')
        if announcement_time > limit_date:
            target_name = i[1].split()[0]

            # 只取還沒資料的
            if dividend_data.get(target_name):
                continue
            else:
                r = requests.get(i[2], headers=header)
                soup = BeautifulSoup(r.text, 'html.parser')

                # 抓取除息資料
                last_buying_day, amount = '', ''
                for tr in soup.select('table tr'):
                    # 最後買進入
                    if '（五）除息交易日' in tr.text:
                        exdividend_trading_day = tr.select_one('td').text
                        exdividend_trading_day = str(int(exdividend_trading_day.split('年')[0]) + 1911) + '年' + exdividend_trading_day.split('年')[1]
                        last_buying_day = datetime.strptime(exdividend_trading_day, '%Y年%m月%d日') - timedelta(days=1)

                    # 配息金額
                    if not amount:
                        for td in tr.select('td'):
                            if search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text):
                                amount = search('(?<=新[臺台]幣) ?\d{1,2}\.\d{1,4}', tr.text).group().strip()

                dividend_data[target_name] = [float(amount), last_buying_day.date()]
            sleep(1+random()*2)

        else:
            break

print(dividend_data)

 46%|████▋     | 46/99 [01:19<01:32,  1.74s/it]

{'大華優利高填息30(00918)': [0.7, datetime.date(2025, 6, 18)], '兆豐龍頭等權重(00921)': [0.12, datetime.date(2025, 6, 16)], '復華台灣科技優息(00929)': [0.07, datetime.date(2025, 6, 17)], '凱基台灣AI50(00952)': [0.02, datetime.date(2025, 6, 16)], '凱基美國非投等債(00945B)': [0.1, datetime.date(2025, 6, 16)], '中信亞太高股息(00964)': [0.043, datetime.date(2025, 6, 16)], '凱基優選高股息30(00915)': [0.21, datetime.date(2025, 6, 16)], '永豐優息存股(00907)': [0.011, datetime.date(2025, 6, 23)], '群益優選非投等債(00953B)': [70.0, datetime.date(2025, 6, 3)], '中信全球高股息(00963)': [0.055, datetime.date(2025, 6, 16)], '中信成長高股息(00934)': [0.028, datetime.date(2025, 6, 16)], '群益科技高息成長(00946)': [0.025, datetime.date(2025, 6, 3)], '中信綠能及電動車(00896)': [0.4, datetime.date(2025, 6, 16)], '台新永續高息中小(00936)': [0.03, datetime.date(2025, 6, 15)], '富邦特選高股息30(00900)': [0.1, datetime.date(2025, 6, 16)], '第一金工業30(00728)': [0.771, datetime.date(2025, 6, 18)], '富邦臺灣優質高息(00730)': [0.11, datetime.date(2025, 6, 16)], '國泰全球品牌50(00916)': [0.25, datetime.date(2025, 6, 16)], '群益台灣精選高息(0

## selenium 爬蟲


In [37]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver

option = webdriver.ChromeOptions() # 設定 chromedriver
# option.add_argument('--headless') # 無頭模式，開發完成之後再使用，可以完全背景執行，有機會變快。但無頭模式在某些網站會不能爬。
option.add_experimental_option('excludeSwitches', ['enable-automation']) # 開發者模式。可以避開某些防爬機制，有開有保佑
driver = webdriver.Chrome(options=option) # 啟動 chromedriver
driver.maximize_window() # 視窗放到最大。有些網站的版面會隨視窗大小改變，在沒有放到最大的情況下有可能定位正確但戳到其他地方，所以通常會放大


### xpath定位
在一些網站使用 css selector 的 id 定位會失敗，可能是網頁架構上故意不給人用 id 定位抓資料，或是其他一些神秘原因，無論如何，如果 css selector 不能用，可以改用 xpath 定位。物件的 xpath 定位跟 css selector 一樣，用開發人員工具在想定位的物件上按右鍵就能找到，選擇 xpath 或 full xpath 皆可。

假設現在要抓研之有物最新文章( https://research.sinica.edu.tw/lastposts/ )第一頁的所有文章標題

In [38]:
url = 'https://research.sinica.edu.tw/lastposts/'
driver.get(url)
driver.implicitly_wait(5)

In [ ]:
body > div.elementor.elementor-13609.elementor-location-archive > section.elementor-section.elementor-top-section.elementor-element.elementor-element-750a24e6.elementor-section-boxed.elementor-section-height-default.elementor-section-height-default > div > div > div > div > div > div.elementor-posts-container.elementor-posts.elementor-posts--skin-classic.elementor-grid.elementor-has-item-ratio > article.elementor-post.elementor-grid-item.post-24674.post.type-post.status-publish.format-standard.has-post-thumbnail.hentry.category-humanities-and-social-sciences.category-view.category-people.tag-1377.tag-1379.tag-1010.tag-1378.tag-1380.home_popular-home-slider > div > h3 > a
body > div.elementor.elementor-13609.elementor-location-archive > section.elementor-section.elementor-top-section.elementor-element.elementor-element-750a24e6.elementor-section-boxed.elementor-section-height-default.elementor-section-height-default > div > div > div > div > div > div.elementor-posts-container.elementor-posts.elementor-posts--skin-classic.elementor-grid.elementor-has-item-ratio > article.elementor-post.elementor-grid-item.post-24613.post.type-post.status-publish.format-standard.has-post-thumbnail.hentry.category-innovation.category-life-sciences.tag-plants.tag-1375.tag-1373.tag-1374.tag-agricultural-biotechnology-research-center.tag-1376.home_popular-home-slider > div > h3 > a
body > div.elementor.elementor-13609.elementor-location-archive > section.elementor-section.elementor-top-section.elementor-element.elementor-element-750a24e6.elementor-section-boxed.elementor-section-height-default.elementor-section-height-default > div > div > div > div > div > div.elementor-posts-container.elementor-posts.elementor-posts--skin-classic.elementor-grid.elementor-has-item-ratio > article.elementor-post.elementor-grid-item.post-24561.post.type-post.status-publish.format-standard.has-post-thumbnail.hentry.category-humanities-and-social-sciences.category-social.category-people.tag-1366.tag-1367.tag-1369.tag-1370.tag-1371.tag-816.tag-1368.home_popular-home-slider > div > h3 > a

進入網頁之後分別定位第一篇、第二篇、最後一篇文章，得到的 xpath 定位如下：
/html/body/div[2]/section[3]/div/div/div/div/div/div[1]/article[1]/div/h3/a
/html/body/div[2]/section[3]/div/div/div/div/div/div[1]/article[2]/div/h3/a
/html/body/div[2]/section[3]/div/div/div/div/div/div[1]/article[12]/div/h3/a

可以觀察出，有變化的只有 article 後面的數字，因此就可以利用 f-string 修改定位規則

In [39]:
title = []
for i in range(1, 13):
    title.append(driver.find_element('xpath', f'/html/body/div[2]/section[3]/div/div/div/div/div/div[1]/article[{i}]/div/h3/a').text)

for i in title:
    print(i)


從醣類到噬菌體：生化學家 Dr. Lowary 的科學探索與人生體悟
為什麼肺結核的細菌很難殺死？解密結核菌的細胞壁「醣」衣
AI 真的有「智慧」嗎？一場探索智慧本質的思辨之旅
翻開冷戰 X 檔案：發掘歷史課本沒寫的冷戰史—專訪陳冠任
綠之護衛隊啟動！植物系統性免疫的超急快遞：訊息傳遞胜肽 CAPE
戴上伊能嘉矩濾鏡：跟著陳偉智看見百年前人類學家的視野（下）
重返伊能嘉矩的踏查之旅！探索臺灣人類學研究的開端（上）
想活下去，你能捨棄演化的墨守成規嗎？酵母菌必需基因的快速演化
誰是拿掃帚的女人？從青銅器解開「婦」字的秘密—專訪內田純子
活在「火焰」上：孔寬楯藤壺與火珊瑚共生的秘密
望向黑洞事件視界：不穩定的發光甜甜圈
奇奇怪怪的黑洞：重力是時空彎曲的體現


### CSS selector 3
實際上，研之有物還是可以用 css selector 定位出最新文章第一頁的所有文章標題，只要會寫就行了。

回顧一下可以打天下的 3 種語法規則：
1. 標籤  p # 抓標籤下的所有層的 p 標籤。
2. 標籤 > p # 抓標籤下一層的所有 p 標籤。
3. 標籤 p:nth-child(n) # 抓標籤下的第 n 個 p 標籤。

標籤可以做限定，比不指定更能縮限範圍，在此只是以class這種屬性舉例，可以指定其他屬性。
1. 標籤[class="字串"] # 抓 class 為指定字串的所有標籤
2. 標籤[class*="字串"] # 抓 class 包含指定字串的所有標籤
3. 標籤[class^="字串"] # 抓 class 以指定字串開頭的所有標籤
4. 標籤[class$="字串"] # 抓 class 以指定字串結尾的所有標籤

In [40]:
# 以研之有物的例子，語法如下：
for i in driver.find_elements('css selector', 'h3 a'):
    print(i.text)

從醣類到噬菌體：生化學家 Dr. Lowary 的科學探索與人生體悟
為什麼肺結核的細菌很難殺死？解密結核菌的細胞壁「醣」衣
AI 真的有「智慧」嗎？一場探索智慧本質的思辨之旅
翻開冷戰 X 檔案：發掘歷史課本沒寫的冷戰史—專訪陳冠任
綠之護衛隊啟動！植物系統性免疫的超急快遞：訊息傳遞胜肽 CAPE
戴上伊能嘉矩濾鏡：跟著陳偉智看見百年前人類學家的視野（下）
重返伊能嘉矩的踏查之旅！探索臺灣人類學研究的開端（上）
想活下去，你能捨棄演化的墨守成規嗎？酵母菌必需基因的快速演化
誰是拿掃帚的女人？從青銅器解開「婦」字的秘密—專訪內田純子
活在「火焰」上：孔寬楯藤壺與火珊瑚共生的秘密
望向黑洞事件視界：不穩定的發光甜甜圈
奇奇怪怪的黑洞：重力是時空彎曲的體現


假設現在要抓取台灣銀行台幣兌美元的最新即期買入匯率( https://rate.bot.com.tw/xrt?Lang=zh-TW )，為了展示 css selector 的定位方式，我們一步一步來：

資料在 tbody 裡面，每一個 tr 都是一列，我們的目標是美元，因此要找第一列，tr 下面有很多個 td，每個 td 都是一欄，即期買入是第 4 欄，因此定位方式就是：

In [41]:
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'
driver.get(url)
driver.implicitly_wait(5)

In [64]:
driver.find_element('css selector', 'tbody tr td:nth-child(4)').text

# css 語法規則的白話文就是，tbody 下面的所有 tr 下面的所有第 4 個 td。
# 但由於是用 find_element，只會找到第一個符合的元素，所以得到的是美元的即期買入匯率。

'30.095'

In [ ]:
# 如果想要的是日圓，語法就改成：
#driver.find_element('css selector', 'tbody tr:nth-child(8) td:nth-child(4)').text
# 或是
driver.find_elements('css selector', 'tbody tr td:nth-child(4)')[7].text


'0.2062'

In [46]:
# 以上只是展示而已，如果真的要抓的話，可以縮減成

driver.find_element('css selector', 'td[data-table="本行即期買入"]').text

# 訣竅在於找出很少重複出現的屬性和屬性值，且盡量不要找看起來像是亂碼的那種

'29.845'

### CSS selector 4
標籤 > p # 抓標籤下一層的所有 p 標籤。

https://www.w3schools.com/cssref/trysel.php

w3schools 是最完整的，但根據經驗常用的只需要會上述講的 3 種就可以打天下

## 下載檔案
有時候爬蟲要抓的是檔案，例如 PDF 或 ZIP，這時候就需要把檔案存起來，舉例來說，現在要用 requests 爬蟲下載 chromedriver，語法跟一般爬蟲幾乎一樣，差別只是爬到的資料不用 BeautifulSoup 解析，而是直接存檔

先進入 https://googlechromelabs.github.io/chrome-for-testing/ ，找到對應自己作業系統的 chromedriver，然後把網址複製貼上到程式碼

In [ ]:
from os import listdir
import requests

chrome_version = listdir(r'C:\Program Files (x86)\Google\Chrome\Application')[0].split('.')[0]
header = {'User-Agent': f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version}.0.0.0 Safari/537.36'}

url = 'https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.55/win64/chromedriver-win64.zip'
r = requests.get(url, headers=header)
with open(url.split('/')[-1], 'wb') as f: # 以二進位(wb)方式存檔，記得要加副檔名
    f.write(r.content) # 不能用 r.text

## 等待連線
如果電腦網路有時候會斷線，讓爬蟲在斷線的時候等待，直到恢復連線

### requests
```python
while i < len(urls):
    try:
        r = requests.get(urls[i], headers=header)
        程式碼
        i += 1
    # 無網路連線時等待連線
    except requests.exceptions.ConnectionError:
        sleep(60)
```
### selenium
WebDriverException 會在瀏覽器版本和 driver 版本不相容、找不到 driver、沒有網路連線的時候出現，必須先排除上述兩種情況，才不會出現在有連線的情況下仍無限等待。由於現在 selenium 會自己管理自己的 driver，不太可能遇到前兩種情況了。
```python
from selenium.common.exceptions import WebDriverException

i = 0
while i < len(urls):
    try:
        driver.get(urls[i])
        程式碼
    # 無網路連線時等待連線
    except WebDriverException:
        sleep(60)
```
### pandas
```python
import urllib

while True:
    try:
        df = pd.read_csv(url)
        break
    # 無網路連線時等待連線
    except urllib.error.URLError:
        sleep(60)
```

## 例行爬蟲
利用 windows 系統的「工作排程器」定期執行 .py 檔。設定簡單歸簡單，有些地雷踩到的話死活不會執行或是直接一閃而過，初期還真不知道怎麼辦。以下就是地雷。

+ 一般

  名稱在排程建立之後就不能改了，要改只能整個刪掉重做一個新的

  安全性選項要選「只有使用者登入時才執行」(選另一個完全不會動)，然後最高權限執行打勾(有些東西要最高權限才能執行，打勾比較不會出問題)

+ 動作

  程式或指令碼輸入「檔名.py」的絕對路徑

  新增引數就是「檔名.py」

  開始位置就是「檔名.py」所在的資料夾的路徑
  
  換句話說，開始位置 + 新增引數 = 程式或指令碼

我個人建議用批次檔(.bat)把 .py 包起來執行，可以把視窗縮到最小不礙事，批次檔語法如下，我對 cmd 也不太熟，可能還可以再改進

```cmd
d:
cd D:\作品集\專案資料夾
start /min call C:\Users\使用者名稱\anaconda3\envs\虛擬環境名稱\python 檔名.py
exit
```

## 驗證碼
圖形驗證碼最開始出現是要防網路攻擊或爬蟲的，因為只要短時間大量送請求就可以癱瘓掉伺服器。

圖形驗證碼發展很長一段時間了，為了讓程式無法突破，中間有一段時間誇張到連人類都看不懂，現在越來越偏向物件辨識(例如 google 的 9 格圖裡面找紅綠燈)、直接點「我不是機器人」(網頁記錄滑鼠的移動軌跡)。甚至是直接監控網頁行為(例如 google 搜尋)，偵測到異常就跳 9 格圖或「我不是機器人」。

老舊的技術出現在老舊的網站，例如政府網站通常都會用圖形驗證碼，但大多很好破，有一點技術的話根本稱不上障礙，商業網站的比較難破。

要輸入文字的驗證碼，如果可以不要走文字辨識，直接抓出字串，成功率可以 100%，但這種狀況應該少見；凡是要用文字辨識的，就有一定的機率辨識錯誤，辨識難度隨著圖片中的干擾物設計而有巨大差異。

目前有 5 種方法可以破解驗證碼，根據某網站驗證碼不進行前處理直接辨識測試 100 張

|作法|說明|正確率|速度|
|:-:|:-:|:-:|:-:|
|TesseractOCR|舊時代光學方法最強的模組|50%|16.4秒|
|easyOCR|基於深度學習的免費 OCR 模組，<br>將多種光學操作做成參數|60%|7.3秒|
|PaddleOCR|基於 LLM 的免費 OCR 模組|14%|1分37秒|
|openAI API|直接請 chatGPT 看|85%|不適用|
|Ollama|本機執行 LLM 讓他看|77%；<br>71%；<br>83%；<br>80%；<br>78%；<br>70%|Mistral-small3.1-24b-instruct-2503-fp16: 10分47秒；<br>Gemma3-27b-it-qat: 4分57秒；<br>Qwen2.5vl-7b-fp16: 3分39秒；<br>Qwen2.5vl-32b-q8_0: 7分48秒；<br>Qwen2.5vl-32b-q4_K_M: 4分7秒；<br>Qwen2.5vl-3b-fp16: 3分39秒|

PaddleOCR 是基於 LLM 的免費 OCR 模組，由中國團隊飛槳開發，安裝這個害我虛擬環境壞掉，辨識驗證碼結果爛到不可思議

Ollama 是以 285HX + 筆電版 5090 運行

|模型|VRAM|RAM|
|:-:|:-:|:-:|
|Mistral-small3.1-24b-instruct-2503-fp16|20.5GB|25.6GB|
|Gemma3-27b-it-qat|19.9GB|2.1GB|
|Qwen2.5vl-7b-fp16|16.8GB|0GB|
|Qwen2.5vl-32b-q8_0|21.5GB|14.8GB|
|Qwen2.5vl-32b-q4_K_M|19.7GB|2.5GB|
|Qwen2.5vl-3b-fp16|9.2GB|1.3GB|

其他測試皆在 8300H + 筆電版 1050 運行

In [3]:
# 沒事不要執行
from os import listdir
import pytesseract

count = 0
files = listdir(r'./驗證碼')
for file in files:
    ans = pytesseract.image_to_string(f'./驗證碼/{file}', lang='eng', config='--psm 7 -c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyz').strip()
    if ans == file.replace('.png', ''):
        count += 1
    print(f'答案:{file.replace(".png", "")} 辨識結果:{ans}')

print(count)

答案:aadq 辨識結果:audi
答案:abza 辨識結果:aba
答案:akyo 辨識結果:akyo
答案:aozw 辨識結果:aoal
答案:auye 辨識結果:cue
答案:begj 辨識結果:hegj
答案:buji 辨識結果:byji
答案:bupy 辨識結果:bupy
答案:caji 辨識結果:caji
答案:celn 辨識結果:celn
答案:cfha 辨識結果:dha
答案:dobf 辨識結果:dob
答案:doya 辨識結果:doya
答案:duxu 辨識結果:duxu
答案:eoeb 辨識結果:edeh
答案:eugb 辨識結果:cb
答案:fahf 辨識結果:fohf
答案:feta 辨識結果:feta
答案:finr 辨識結果:finp
答案:fowa 辨識結果:fowa
答案:gcvb 辨識結果:ge
答案:gfla 辨識結果:gfla
答案:gjne 辨識結果:gine
答案:gupl 辨識結果:gupl
答案:gura 辨識結果:gra
答案:hbxe 辨識結果:hbxe
答案:hepj 辨識結果:hey
答案:hime 辨識結果:hime
答案:hire 辨識結果:ire
答案:iive 辨識結果:ive
答案:ioxi 辨識結果:joxi
答案:iuki 辨識結果:iki
答案:jauw 辨識結果:jouw
答案:jedi 辨識結果:jedi
答案:jobu 辨識結果:jobu
答案:jopi 辨識結果:jopi
答案:juai 辨識結果:juai
答案:juux 辨識結果:jeux
答案:kaeu 辨識結果:kaeu
答案:kise 辨識結果:kise
答案:kqlo 辨識結果:kqb
答案:lbsi 辨識結果:lbsi
答案:leto 辨識結果:leto
答案:lipi 辨識結果:lipi
答案:loto 辨識結果:loto
答案:lpvk 辨識結果:ipvk
答案:maha 辨識結果:maha
答案:maxi 辨識結果:maxi
答案:mume 辨識結果:mume
答案:neom 辨識結果:nom
答案:neru 辨識結果:neru
答案:nize 辨識結果:nize
答案:nkwe 辨識結果:nkwe
答案:nudi 辨識結果:nudi
答案:obvn 辨識結果:obun
答案:pami 辨識結果:pami
答案:papl

In [1]:
# 匯入模組
# 每次匯入都需要一段時間，因此獨立出來避免重複運行
from os import listdir
import easyocr

In [2]:
# 辨識看看吧
reader = easyocr.Reader(['en'])

count = 0
files = listdir(r'./驗證碼')
for file in files:
    ans = reader.readtext(f'./驗證碼/{file}', allowlist='abcdefghijklmnopqrstuvwxyz', detail=0, paragraph=True)[0]
    if ans == file.replace('.png', ''):
        count += 1
    print(f'答案:{file.replace(".png", "")} 辨識結果:{ans}')

print(count)

答案:aadq 辨識結果:aadq
答案:abza 辨識結果:dbza
答案:akyo 辨識結果:akyo
答案:aozw 辨識結果:aozn
答案:auye 辨識結果:duye
答案:begj 辨識結果:begj
答案:buji 辨識結果:bji
答案:bupy 辨識結果:bpy
答案:caji 辨識結果:aji
答案:celn 辨識結果:celn
答案:cfha 辨識結果:dha
答案:dobf 辨識結果:dl
答案:doya 辨識結果:dya
答案:duxu 辨識結果:duxu
答案:eoeb 辨識結果:edeb
答案:eugb 辨識結果:eugb
答案:fahf 辨識結果:fahf
答案:feta 辨識結果:feta
答案:finr 辨識結果:fnp
答案:fowa 辨識結果:fond
答案:gcvb 辨識結果:gcvb
答案:gfla 辨識結果:gfla
答案:gjne 辨識結果:gne
答案:gupl 辨識結果:gupl
答案:gura 辨識結果:gura
答案:hbxe 辨識結果:hbxe
答案:hepj 辨識結果:hed
答案:hime 辨識結果:hime
答案:hire 辨識結果:hire
答案:iive 辨識結果:ie
答案:ioxi 辨識結果:ioxi
答案:iuki 辨識結果:iuki
答案:jauw 辨識結果:jauw
答案:jedi 辨識結果:jedi
答案:jobu 辨識結果:joby
答案:jopi 辨識結果:jopi
答案:juai 辨識結果:juai
答案:juux 辨識結果:jvux
答案:kaeu 辨識結果:kaeu
答案:kise 辨識結果:kise
答案:kqlo 辨識結果:kqb
答案:lbsi 辨識結果:lbsi
答案:leto 辨識結果:leto
答案:lipi 辨識結果:lipi
答案:loto 辨識結果:loto
答案:lpvk 辨識結果:lpvk
答案:maha 辨識結果:maha
答案:maxi 辨識結果:maxi
答案:mume 辨識結果:nne
答案:neom 辨識結果:neom
答案:neru 辨識結果:nerv
答案:nize 辨識結果:nize
答案:nkwe 辨識結果:nkwe
答案:nudi 辨識結果:nudi
答案:obvn 辨識結果:cbm
答案:pami 辨識結果:pami
答案:papl

In [4]:
# 沒事不要執行
from os import listdir
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')
count = 0
files = listdir(r'./驗證碼')
for file in files:
    ans = ocr.ocr(f'./驗證碼/{file}')
    if ans[0]['rec_texts']:
        if ans[0]['rec_texts'][0] == file.replace('.png', ''):
            count += 1
    print(f"答案:{file.replace('.png', '')} 辨識結果: {ans[0]['rec_texts'][0] if ans[0]['rec_texts'] else ''}, 信心分數: {ans[0]['rec_scores'][0] if ans[0]['rec_scores'] else ''}")

print(count)

C:\Users\hchbi\AppData\Local\Temp\ipykernel_20160\3980911226.py:5: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang='en')
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\hchbi\.paddlex\official_models.
c:\Users\hchbi\anaconda3\envs\General\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\hchbi\.paddlex\official_models.
Creating model: ('PP-LCNet_x0_2

答案:aadq 辨識結果: aada, 信心分數: 0.8958593010902405
答案:abza 辨識結果: ulbza, 信心分數: 0.688918948173523
答案:akyo 辨識結果: , 信心分數: 
答案:aozw 辨識結果: Uozw, 信心分數: 0.5248073935508728
答案:auye 辨識結果: , 信心分數: 0.0
答案:begj 辨識結果: C, 信心分數: 0.28321826457977295
答案:buji 辨識結果: , 信心分數: 
答案:bupy 辨識結果: -, 信心分數: 0.1906553953886032
答案:caji 辨識結果: C, 信心分數: 0.7683184146881104
答案:celn 辨識結果: q, 信心分數: 0.23881712555885315
答案:cfha 辨識結果: I, 信心分數: 0.4739779531955719
答案:dobf 辨識結果: , 信心分數: 0.0
答案:doya 辨識結果: pfop, 信心分數: 0.8489751219749451
答案:duxu 辨識結果: , 信心分數: 
答案:eoeb 辨識結果: , 信心分數: 
答案:eugb 辨識結果: eu, 信心分數: 0.7638556957244873
答案:fahf 辨識結果: , 信心分數: 0.0
答案:feta 辨識結果: •, 信心分數: 0.14730624854564667
答案:finr 辨識結果: , 信心分數: 
答案:fowa 辨識結果: FaWa, 信心分數: 0.6157865524291992
答案:gcvb 辨識結果: , 信心分數: 
答案:gfla 辨識結果: , 信心分數: 
答案:gjne 辨識結果: 6, 信心分數: 0.967566728591919
答案:gupl 辨識結果: , 信心分數: 0.0
答案:gura 辨識結果: gura, 信心分數: 0.9892891645431519
答案:hbxe 辨識結果: つ, 信心分數: 0.8665969967842102
答案:hepj 辨識結果: , 信心分數: 0.0
答案:hime 辨識結果: hime, 信心分數: 0.9870278239250183
答案:hire 辨識結果:

# 爬蟲的種類
+ 個人爬蟲：自己寫的
+ 搜尋引擎爬蟲：搜尋引擎用的
+ LLM 爬蟲：抓到的資料用來訓練 LLM
+ 預覽爬蟲：各家軟體公司給自己客戶用的，貼上連結後會有預覽圖和文字，同時會進行資安審查
+ 檔案館爬蟲：有些公司會把網路上的網頁都備份下來，作為檔案館
+ SEO 爬蟲：抓取目標網站的網頁結構、連結結構、測試網頁效能，用於分析目標網站的 SEO 成效

## 搜尋引擎
搜尋引擎爬蟲定期以廣度優先搜尋遍歷目標網站，但凡看到網頁中有連結就都點開，下載網頁原始碼和圖片存進資料庫並製作索引。

當使用者在搜尋引擎輸入關鍵字，搜尋引擎就會去資料庫尋找有相關資料的網頁，並根據相關性排序，最後呈現給使用者。

較有名的搜尋引擎是 google 和 Bing，搜尋引擎好不好用，取決於搜尋引擎爬蟲的功力，例如效率越高的爬蟲，就可以越頻繁地更新網頁資料

### robots.txt
搜尋引擎爬蟲遵守各網站的 robots.txt 的規範(例如 https://shopee.tw/robots.txt )，不允許爬的就不會去爬，私人爬蟲完全可以不用管這個，也不是所有網站都會有這個檔案。

robots.txt 的內容就是說哪些機器人可以和不可以爬哪些網頁，但由於只是宣告性質，真正的防爬機制還是會做。

## LLM 連網能力
開發 LLM 的公司也會寫爬蟲從網路上抓資料用來訓練 LLM，這類爬蟲理論上也會遵守 robots.txt，但只要他最後不公開資料集其實你也不知道他有沒有違反。

LLM 連網的能力有兩種：

1. LLM 在使用者提問之後立刻去看網頁
    + 很即時，但只能看 GET 就能得到所有資料的網頁，凡是較晚才補資料、需要 POST、需要登入的他都不能看

2. LLM 使用傳統搜尋引擎的資料庫，將其當作參考資料來源回答，這是一種 RAG，資料有多新、多完整取決於使用的搜尋引擎爬蟲能力有多強
    + chatGPT 用的是 Bing，Gemini 用的是 google。在 ETF 最新訊息的測試，Gemini 可以完全答對正是因為其自家功力深厚
    + 所有防爬機制對於傳統搜尋引擎爬蟲來說都是可攻破的，那些真正不可攻破又持有大量流量的網站(例如華盛頓郵報)，反而可以跟搜尋引擎公司開價，「給多少就讓你爬」，例如 openAI 就有跟華盛頓郵報買( https://technews.tw/2025/04/23/technews-ai-20250423-57946/ )。
    + LLM 可以使用傳統搜尋引擎爬蟲的成果來回答問題，網頁流量將會不可避免地減少

理想上，兩種連網能力解決大部分 LLM 資料過期的問題，但就算資料正確，還是有瞎掰的問題(例如 Grok )

# AI 爬蟲的極限
比 GET 難抓的，你自己寫都比給他抓還要好

但 LLM 很會寫程式，因此可以請他協助你寫爬蟲抓資料

結論：用解答者製造影分身

# 爬蟲的智慧財產權問題
爬蟲就只是自動抓資料，技術好的話還可以抓到對方不太想給的資料，但這並不代表抓到的資料可以任意使用。例如購物網站的圖片，版權是屬於網站的，如果你抓了之後直接用在自己的服務上，對方可以告你侵權，例如 openAI 被紐約時報告 ( https://www.sbir.org.tw/ipcc/news_content?id=11481&page=1 )。

較好的爬蟲專案
+ 低調，他們想從伺服器記錄撈你的 IP 也撈不出來、你登入的帳號看不出有什麼異常行為
+ 自己用，沒有人受害
+ 不直接把抓到的資料用在自己的服務上，整理分析之後只呈現分析結果就好
+ 一定要用圖的話，自己重畫


# 好用模組推薦
建議用複製的，一字不差才可以保證搜到正確的東西

+ 時間類
  + datetime: 處理時間資料，內建
+ 數學類
  + math: 數學，內建
  + random: 隨機，內建
+ 檔案類
  + os: 查詢檔案、編輯檔案的名稱及路徑、刪除檔案，內建
  + shutil: 複製、刪除、移動檔案和資料夾，內建
+ 文書類
  + python-docx: 控制 word
  + openpyxl、xlwings: 控制 Excel。據說 openpyxl 比較好
  + pygsheets: 控制 google sheet
  + PyMuPdf: 讀取 pdf 檔
  + pyautogui: 鍵鼠模擬，可以用來寫類似按鍵精靈的外掛自動操作，會失去控制權
  + zipfile: 壓縮/解壓縮 zip 檔，內建
  + py7zr: 壓縮/解壓縮 7z 檔
+ 介面類
  + tkinter: 使用者介面，陽春，內建，不用會class
  + PyQt6: 使用者介面，精緻，需要會class
  + gradio: 使用者介面，呈現在瀏覽器上
  + streamit: ??
+ 通訊類
  + SSL + smtplib + email: 寄信，三個模組皆為內建
  + LINE bot: 用 LINE 收發訊息
  + Telegram bot: 用 Telegram 送通知
+ 自然語言處理
  + ollama: 本機 LLM
  + openai: 使用 openai 的 API
  + 情感分析: transformers + HuggingFace 上的模型
+ 爬蟲類
  + requests: 收發請求
  + selenium: 網頁測試
  + BeautifulSoup: 解析網頁原始碼
+ 資料處理
  + json: 讀寫 json 格式，內建
  + re: 正規表達式，尋找特定模式的字串，內建
  + pandas: 資料前處理
+ 科學計算
  + scipy: 線性代數、最佳化、統計、找峰值，很多就對了
  + statsmodels: 統計
+ 視覺化
  + Matplotlib、Seaborn: 畫統計圖。Seaborn 是 Matplotlib 的延伸，比較漂亮
  + Plotly: 畫可以互動的圖
  + folium: 在地圖上做標記
+ 影像處理
  + openCV: 影像處理該有的都有
  + easyOCR: 文字辨識
  + YOLOv12: 物件辨識
+ 網頁
  + flask: 輕量級網頁框架
  + Django 5: 全端網頁框架，還需要學 HTML、CSS、JavaScript
  + fastAPI: 後端框架
+ 手機 APP
  + Kivy: 打包成 apk 檔，安卓可以用，iPhone 不能用

# 其他學習資源
+ STEAM教育學習網 https://steam.oxxostudio.tw/category/python/index.html
  + 有很多模組教學，但不一定是最好的寫法，文章寫成之後也不一定有更新
+ 我阿嬤都會 https://www.youtube.com/@GrandmaCan/videos
  + 用很白話的方式教寫程式，有些主題要完整的內容要付費
+ 李宏毅 https://www.youtube.com/@HungyiLeeNTU/videos
  + 台大電機副教授，每年都會開機器學習課程，且課程內容都會放上個人網頁 https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php
+ 書籍資料
  + 電腦領域的事情更新很快，基本上就是越新越好
  + 但參考價值還是要自己判斷，有些事情不太會隨著時間變動
  + 各縣市圖書館、國家圖書館、台灣雲端書庫( https://www.ebookservice.tw/ )

# 學期結束之後

我們團隊目標之一是讓學會 python 的人可以把這項技術運用在生活或工作中，把生活或工作中的一些事情自動化，提升效率、增加可用時間、減少壓力。如果你想在工作使用 python

1. 低調，小心能者過勞
2. 只讓可信任的人知道

如果你想變強

1. 上面是我推薦的自學資源
2. 報名加入我們團隊
3. 敬請期待校友活動